# Análise de dados do Ar

## Fonte dos dados
[Air Quality Open Data Platform](https://aqicn.org/data-platform/covid19/)

The data for each major cities is based on the average (median) of several stations. The data set provides min, max, median and standard deviation for each of the air pollutant species (PM2.5,PM10, Ozone ...) as well as meteorological data (Wind, Temperature, ...). All air pollutant species are converted to the US EPA standard (i.e. no raw concentrations). All dates are UTC based. The count column is the number of samples used for calculating the median and standard deviation.

In [1]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import json
pd.set_option('display.max_columns', None)

df = pd.read_csv('../data/air-quality/results/iqar-continents.csv')
df_index = pd.read_csv('../data/air-quality/classification-iqar.csv')

In [2]:
df_index

,INDEX,QUALITY,pm25,pm10,so2,no2,co,o3
0,0,BOA,25,50,20,200,9.00,100
1,1,REGULAR,60,120,125,260,9.05,140
2,2,INADEQUADA,124,249,799,1129,14.90,199
3,3,RUIM,209,419,1599,2259,29.90,399
4,4,PESSIMO,249,499,2099,2999,39.90,599


In [3]:
df

,DATE,REGION,SPECIE,MEAN_BY_CONTINENT
0,2014,Africa,no2,2.633333
1,2014,Africa,o3,10.333333
2,2014,Africa,pm10,10.166667
3,2014,Africa,pm25,16.333333
4,2014,Africa,so2,1.750000
...,...,...,...,...
263,2022,Oceania,no2,3.591001
264,2022,Oceania,o3,10.574130
265,2022,Oceania,pm10,11.502125
266,2022,Oceania,pm25,21.930735


In [4]:
df_index[df_index['no2'] > 2.633333].head(1)
df_index[df_index['o3'] >= 700].head(1)["INDEX"]

Series([], Name: INDEX, dtype: int64)

In [5]:
def search_quality(row):
    #print(row)
    specie = row["SPECIE"]
    mean = row["MEAN_BY_CONTINENT"]
    filtered = df_index[df_index[specie] >= mean]
    if(filtered.empty): return 5
    return filtered.iloc[0]['INDEX']

df["CLASSIFICATION"] = df.apply(search_quality, axis=1)
df.head(10)

,DATE,REGION,SPECIE,MEAN_BY_CONTINENT,CLASSIFICATION
0,2014,Africa,no2,2.633333,0
1,2014,Africa,o3,10.333333,0
2,2014,Africa,pm10,10.166667,0
3,2014,Africa,pm25,16.333333,0
4,2014,Africa,so2,1.750000,0
5,2014,Americas,co,6.358942,0
6,2014,Americas,no2,8.751465,0
7,2014,Americas,o3,10.889843,0
8,2014,Americas,pm10,28.594598,0
9,2014,Americas,pm25,53.205987,1


In [6]:
df[df["CLASSIFICATION"] != 0]

,DATE,REGION,SPECIE,MEAN_BY_CONTINENT,CLASSIFICATION
9,2014,Americas,pm25,53.205987,1
11,2014,Asia,co,9.349231,2
14,2014,Asia,pm10,60.019908,1
15,2014,Asia,pm25,115.517080,2
21,2014,Europe,pm25,73.747536,2
27,2014,Oceania,pm25,25.222222,1
39,2015,Americas,pm25,64.241559,2
45,2015,Asia,pm25,83.961860,2
51,2015,Europe,pm25,45.549463,1
57,2015,Oceania,pm25,28.152625,1


In [7]:
df = df.groupby(['DATE', 'REGION']).mean().reset_index()

In [8]:
df.to_csv('../data/air-quality/results/iqar-continents-class-after.csv',index=False)